In [20]:
# day 11 - part 1
with open('advent-of-code-day11-input', 'r') as file:
# with open('advent-of-code-day11-testcase', 'r') as file:
  input_data = file.read()

# split the input data by "\n"
input_data = input_data.split("\n")

# filter from empty elements
input_data = list(filter(None, input_data))

# check (good)
# input_data

# name it good
image = input_data
# image

In [14]:
# helper transpose function
def transpose(matrix):
  return [list(col) for col in zip(*matrix)] # col in A becomes row here

# define a function that expands the space
def expandSpace(image):
  image = [list(row) for row in image] # must be done to not make it change inplace, something weird with strings
  emptyRows, emptyCols = [], []
  for i, row in enumerate(image): # iterate over rows
    if all(space == '.' for space in row):
      emptyRows.append(i)

  for j, col in enumerate(transpose(image)): # iterate over cols
    if all(space == '.' for space in col):
      emptyCols.append(j)

  # emptyRows and emptyCols will change dynamically when the image is inserted, so this needs adjustment
  emptyRows = [(emptyRow, emptyRow + i) for i, emptyRow in enumerate(emptyRows)]
  emptyCols = [(emptyCol, emptyCol + i) for i, emptyCol in enumerate(emptyCols)]

  # now insert the empty rows and cols
  for _, emptyRow in emptyRows:
    image.insert(emptyRow, image[emptyRow])

  image = transpose(image) # transpose for cols
  for _, emptyCol in emptyCols:
    image.insert(emptyCol, image[emptyCol])

  image = transpose(image) # transpose again to get to the original orientation
  image = [''.join(row) for row in image]

  return image
# expandSpace(image) # good

# helper function
def distanceGalaxyPair(galA, galB):
  distI = abs(galA[0] - galB[0])
  distJ = abs(galA[1] - galB[1])
  return distI + distJ

def galaxyFinder(image):
  galaxies = []
  for i, row in enumerate(image):
    for j, el in enumerate(row):
      if el == "#":
        galaxies.append((i, j))
  return galaxies

from itertools import combinations
# now define a function that returns distance to every pair of galaxies there is
def sumDistanceGalaxyPairs(image):
  image = [list(row) for row in image] # avoid immutable strings shenanigans
  galaxies = galaxyFinder(image) # find galaxies
  galaxiesCombination = list(combinations(galaxies, 2)) # find all possible galaxy combinations

  total = 0
  for galA, galB in galaxiesCombination:
    total += distanceGalaxyPair(galA, galB)

  return total

sumDistanceGalaxyPairs(expandSpace(image))
# 374, right for the testcase
# 9684228 RIGHT first try
# finished in 59'

374

In [21]:
# day 11 - part 2
with open('advent-of-code-day11-input', 'r') as file:
# with open('advent-of-code-day11-testcase', 'r') as file:
  input_data = file.read()

# split the input data by "\n"
input_data = input_data.split("\n")

# filter from empty elements
input_data = list(filter(None, input_data))

# check (good)
# input_data

# name it good
image = input_data
# image

In [22]:
# hmmm i think i should just find emptyRows and Cols, then jsut find clever ways to sum over it?
def emptyRowsCols(image):
  image = [list(row) for row in image] # must be done to not make it change inplace, something weird with strings
  emptyRows, emptyCols = [], []

  for i, row in enumerate(image): # iterate over rows
    if all(space == '.' for space in row):
      emptyRows.append(i)

  for j, col in enumerate(transpose(image)): # iterate over cols
    if all(space == '.' for space in col):
      emptyCols.append(j)

  return emptyRows, emptyCols

# redefine the dist function
def distanceGalaxyPairV2(galA, galB, emptyRows, emptyCols, multiplier):
  maxRow = max(galA[0], galB[0])
  minRow = min(galA[0], galB[0])
  maxCol = max(galA[1], galB[1])
  minCol = min(galA[1], galB[1])

  distRow = maxRow - minRow
  distCol = maxCol - minCol

  emptyRowsTraversed = sum([1 for row in range(minRow+1, maxRow) if row in emptyRows])*(multiplier-1)
  emptyColsTraversed = sum([1 for col in range(minCol+1, maxCol) if col in emptyCols])*(multiplier-1) # this is a bit peculiar but think about it

  return distRow + distCol + emptyRowsTraversed + emptyColsTraversed

def sumDistanceGalaxyPairsV2(image, multiplier):
  image = [list(row) for row in image] # avoid immutable strings shenanigans
  emptyRows, emptyCols = emptyRowsCols(image) # find all empty rows and cols for distance calc
  galaxies = galaxyFinder(image) # find galaxies
  galaxiesCombination = list(combinations(galaxies, 2)) # find all possible galaxy combinations

  total = 0
  for galA, galB in galaxiesCombination:
    total += distanceGalaxyPairV2(galA, galB, emptyRows, emptyCols, multiplier)

  return total

sumDistanceGalaxyPairsV2(image, 10**6)
# 374 right for the first testcase
# 1030, right w/ test mul 10
# 8410, right w/ test mul 100
# 483844716556, RIGHT FIRST TRY!!
# took me 31' lfg

483844716556

# scratchpad

In [ ]:
### FIRST ITERATION

# part 2
# this is wrong
# redefine expandSpace then find the dist as it is?
def expandSpaceMultiplied(image, multiplier):
  image = [list(row) for row in image] # must be done to not make it change inplace, something weird with strings
  emptyRows, emptyCols = [], []
  for i, row in enumerate(image): # iterate over rows
    if all(space == '.' for space in row):
      emptyRows.append(i)

  for j, col in enumerate(transpose(image)): # iterate over cols
    if all(space == '.' for space in col):
      emptyCols.append(j)

  # emptyRows and emptyCols will change dynamically when the image is inserted, so this needs adjustment
  emptyRows = [(emptyRow, emptyRow + i*(multiplier-1)) for i, emptyRow in enumerate(emptyRows)]
  emptyCols = [(emptyCol, emptyCol + i*(multiplier-1)) for i, emptyCol in enumerate(emptyCols)]

  # now insert the empty rows and cols
  for _, emptyRow in emptyRows:
    image = image[:emptyRow] + [image[emptyRow]]*(multiplier-1) + image[emptyRow:]

  image = transpose(image) # transpose for cols
  for _, emptyCol in emptyCols:
    image = image[:emptyCol] + [image[emptyCol]]*(multiplier-1) + image[emptyCol:]

  image = transpose(image) # transpose again to get to the original orientation
  image = [''.join(row) for row in image]

  return image

expandSpaceMultiplied(image, 10) # good
sumDistanceGalaxyPairs(expandSpaceMultiplied(image, 10))
# 1030, right for testcase w/ multiplier of 10
# 8410, right for testcase w/ multiplier of 100
# right for all testcases but RAM crashed when i tried to expandSpaceMultiplied with 10**6 (a million) lmao

In [7]:
for i in range(6,3):
  print(i)

3
4
5
